In [ ]:
#import os
#os.makedirs("bin2cell/output/P2CRC/stardist", exist_ok=True)

SPHERE on real data

1. scaled_he_image + stardist（H&E 核分割）
2. expand_labels（核到细胞边界扩张）
2. grid_image + stardist（GEX细胞范围分割）
4. salvage_secondary_labels（融合H&E和GEX分割结果）
5. bin_to_cell（关键步骤：得到 cell × gene 矩阵）

In [1]:
import matplotlib.pyplot as plt
import scanpy as sc
import bin2cell as b2c

# ==== 数据路径 ====
path = "/home/wangzhuo/data/Visium_HD_Human_Colon_Cancer/P2CRC/binned_outputs/square_002um/"
source_image_path ="/home/wangzhuo/data/Visium_HD_Human_Colon_Cancer/P2CRC/Visium_HD_Human_Colon_Cancer_P2_tissue_image.btf"
spaceranger_image_path = "/home/wangzhuo/data/Visium_HD_Human_Colon_Cancer/P2CRC/spatial"

# ==== 1. 读取 VisiumHD 数据 ====
adata = b2c.read_visium(
    path,
    source_image_path=source_image_path,
    spaceranger_image_path=spaceranger_image_path
)
adata.var_names_make_unique()

# 基础过滤（可选）
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_counts=1)

# 设置分辨率（mpp: microns per pixel）
mpp = 0.5

# ==== 2. 生成 H&E 缩放图，供 Stardist 分割用 ====
b2c.scaled_he_image(adata, mpp=mpp, save_path="bin2cell/output/P2CRC/stardist/he.tiff")
b2c.destripe(adata)

# ==== 3. 细胞核分割（H&E） ====
'''b2c.stardist(image_path="bin2cell/output/P2CRC/stardist/he.tiff", 
             labels_npz_path="bin2cell/output/P2CRC/stardist/he.npz", 
             stardist_model="2D_versatile_he", 
             prob_thresh=0.01
            )'''
b2c.insert_labels(adata, 
                  labels_npz_path="bin2cell/output/P2CRC/stardist/he.npz", 
                  basis="spatial", 
                  spatial_key="spatial_cropped_150_buffer",
                  mpp=mpp, 
                  labels_key="labels_he"
                 )

# ==== 4. 扩张核标签到细胞边界（pseudo cell mask） ====
b2c.expand_labels(
    adata,
    labels_key='labels_he',
    expanded_labels_key="labels_he_expanded"
)

# ==== 5. 基因信号图（用 n_counts_adjusted 生成） ====
b2c.grid_image(adata, "n_counts_adjusted", mpp=mpp, sigma=5, save_path="bin2cell/output/P2CRC/stardist/gex.tiff")

# ==== 6. 细胞质分割（基因信号） ====
'''b2c.stardist(image_path="bin2cell/output/P2CRC/stardist/gex.tiff", 
             labels_npz_path="bin2cell/output/P2CRC/stardist/gex.npz", 
             stardist_model="2D_versatile_fluo", 
             prob_thresh=0.05, 
             nms_thresh=0.5
            )'''
b2c.insert_labels(adata, 
                  labels_npz_path="bin2cell/output/P2CRC/stardist/gex.npz", 
                  basis="array", 
                  mpp=mpp, 
                  labels_key="labels_gex"
                 )

# ==== 7. 合并核分割 + 质分割结果 ====
b2c.salvage_secondary_labels(
    adata,
    primary_label="labels_he_expanded",
    secondary_label="labels_gex",
    labels_key="labels_joint"
)

# ==== 8. Bin → Cell 分配表达矩阵 ====
'''cdata = b2c.bin_to_cell(
    adata,
    labels_key="labels_joint",
    spatial_keys=["spatial", "spatial_cropped_150_buffer"]
)'''

# 此时 cdata 就是 cell × gene 的表达矩阵
# cdata.obs 包含每个 cell 的 ID / 位置信息
# cdata.var 是基因信息

#print(cdata)

anndata.py (1820): Variable names are not unique. To make them unique, call `.var_names_make_unique`.
anndata.py (1820): Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Cropped spatial coordinates key: spatial_cropped_150_buffer
Image key: 0.5_mpp_150_buffer


_construct.py (148): Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


Salvaged 8584 secondary labels


'cdata = b2c.bin_to_cell(\n    adata,\n    labels_key="labels_joint",\n    spatial_keys=["spatial", "spatial_cropped_150_buffer"]\n)'

In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
from shapely.validation import make_valid
import matplotlib.pyplot as plt


# ==== 1. 自动找ROI ====
def find_high_expr_roi(adata, gene_name, win_row=100, win_col=100):
    expr = np.ravel(adata[:, gene_name].X.toarray())
    rows = adata.obs['array_row'].values
    cols = adata.obs['array_col'].values
    row_min, row_max = rows.min(), rows.max()
    col_min, col_max = cols.min(), cols.max()

    best_score = -np.inf
    best_roi = None
    for r0 in range(row_min, row_max - win_row, win_row // 2):
        for c0 in range(col_min, col_max - win_col, win_col // 2):
            r1 = r0 + win_row
            c1 = c0 + win_col
            mask = (rows >= r0) & (rows <= r1) & (cols >= c0) & (cols <= c1)
            if np.sum(mask) < 10:
                continue
            score = expr[mask].mean()
            if score > best_score:
                best_score = score
                best_roi = (r0, r1, c0, c1)
    return best_roi, best_score


# ==== 2. 三种方法 ====
def naive_assignment(bin_gdf, cell_gdf):
    sj = gpd.sjoin(bin_gdf, cell_gdf, how="inner", predicate="within")
    genes = [c for c in bin_gdf.columns if c != "geometry"]
    sj["cell_id"] = sj["index_right"].map(cell_gdf["cell_id"])
    return sj.groupby("cell_id")[genes].sum()


def weighted_by_area(bin_gdf, cell_gdf):
    genes = [c for c in bin_gdf.columns if c != "geometry"]
    sj = gpd.sjoin(bin_gdf, cell_gdf, how="inner", predicate="intersects")
    sj["overlap_area"] = sj.apply(
        lambda r: r["geometry"].intersection(cell_gdf.loc[r["index_right"], "geometry"]).area, axis=1)
    sj["bin_area"] = sj["geometry"].area
    sj["weight"] = sj["overlap_area"] / sj["bin_area"]
    weighted = sj[genes].multiply(sj["weight"], axis=0)
    weighted["cell_id"] = sj["index_right"].map(cell_gdf["cell_id"])
    return weighted.groupby("cell_id")[genes].sum()


def sphere_assignment(bin_gdf, cell_gdf, alpha=0.01):
    genes = [c for c in bin_gdf.columns if c != "geometry"]
    sj = gpd.sjoin(bin_gdf, cell_gdf, how="inner", predicate="intersects")
    sj["overlap_area"] = sj.apply(
        lambda r: r["geometry"].intersection(cell_gdf.loc[r["index_right"], "geometry"]).area, axis=1)
    sj["bin_area"] = sj["geometry"].area
    sj["area_weight"] = sj["overlap_area"] / sj["bin_area"]

    sj["bin_cx"] = sj["geometry"].centroid.x
    sj["bin_cy"] = sj["geometry"].centroid.y
    cell_gdf["cx"] = cell_gdf["geometry"].centroid.x
    cell_gdf["cy"] = cell_gdf["geometry"].centroid.y
    sj["cell_cx"] = sj["index_right"].map(cell_gdf["cx"])
    sj["cell_cy"] = sj["index_right"].map(cell_gdf["cy"])

    sj["dist_weight"] = np.exp(
        -alpha * np.sqrt((sj["bin_cx"] - sj["cell_cx"]) ** 2 + (sj["bin_cy"] - sj["cell_cy"]) ** 2)
    )
    sj["raw_w"] = sj["area_weight"] * sj["dist_weight"]
    sj["weight"] = sj.groupby(sj.index)["raw_w"].transform(lambda x: x / x.sum())

    weighted = sj[genes].multiply(sj["weight"], axis=0)
    weighted["cell_id"] = sj["index_right"].map(cell_gdf["cell_id"])
    return weighted.groupby("cell_id")[genes].sum()


# ==== 3. 绘图+统计（已改好类型判断）====
def plot_gene_roi_comparison_with_stats(cell_gdf, pred_naive, pred_wa, pred_sphere,
                                        gene, boundary_q=0.6, savepath=None):
    methods = {
        "Naïve": pred_naive,
        "Weighted Area": pred_wa,
        "SPHERE": pred_sphere
    }
    vmin = 0
    vmax = max(pred_naive[gene].max(), pred_wa[gene].max(), pred_sphere[gene].max())

    # ROI 高表达 hull
    mask_ids = pred_sphere.index[pred_sphere[gene] > pred_sphere[gene].quantile(boundary_q)]
    gland_cells = cell_gdf[cell_gdf["cell_id"].isin(mask_ids)]
    gland_hull = MultiPoint([geom.centroid for geom in gland_cells.geometry]).convex_hull

    fig, axes = plt.subplots(1, 4, figsize=(22, 6))

    for ax, (name, expr_df) in zip(axes[:3], methods.items()):
        aligned_expr = expr_df.reindex(cell_gdf["cell_id"]).fillna(0)
        plot_gdf = cell_gdf.assign(expr=aligned_expr[gene].values)
        plot_gdf.plot(column="expr", cmap="viridis", legend=True, ax=ax,
                      edgecolor="black", linewidth=0.2, vmin=vmin, vmax=vmax)

        if isinstance(gland_hull, (Polygon, MultiPolygon)):
            gx, gy = gland_hull.exterior.xy
            ax.plot(gx, gy, color="red", linewidth=1.5)

        ax.set_title(f"{gene} - {name}", fontsize=14)
        ax.axis('off')

    # 差值图
    aligned_sphere = pred_sphere.reindex(cell_gdf["cell_id"]).fillna(0)
    aligned_wa = pred_wa.reindex(cell_gdf["cell_id"]).fillna(0)
    diff_series = aligned_sphere[gene] - aligned_wa[gene]
    plot_gdf_diff = cell_gdf.assign(diff=diff_series.values)
    plot_gdf_diff.plot(column="diff", cmap="bwr", legend=True, ax=axes[3],
                       vmin=-abs(diff_series).max(), vmax=abs(diff_series).max(),
                       edgecolor="black", linewidth=0.2)

    if isinstance(gland_hull, (Polygon, MultiPolygon)):
        gx, gy = gland_hull.exterior.xy
        axes[3].plot(gx, gy, color="red", linewidth=1.5)

    axes[3].set_title(f"{gene}: SPHERE - WA", fontsize=14)
    axes[3].axis('off')

    plt.tight_layout()
    if savepath:
        plt.savefig(f"{savepath}.png", dpi=300)
    plt.close()

    # 统计
    if isinstance(gland_hull, (Polygon, MultiPolygon)):
        gland_ids_in_hull = cell_gdf[cell_gdf.geometry.centroid.within(gland_hull)]["cell_id"]
    else:
        gland_ids_in_hull = []

    stats = {}
    for name, expr_df in methods.items():
        aligned_expr = expr_df.reindex(cell_gdf["cell_id"]).fillna(0)
        if len(gland_ids_in_hull) > 0:
            stats[name] = aligned_expr.loc[gland_ids_in_hull, gene].mean()
        else:
            stats[name] = np.nan

    if not np.isnan(stats["Weighted Area"]) and not np.isnan(stats["SPHERE"]) and stats["Weighted Area"] != 0:
        improvement = (stats["SPHERE"] - stats["Weighted Area"]) / stats["Weighted Area"] * 100
    else:
        improvement = np.nan

    return len(gland_ids_in_hull), stats, improvement


# ==== 4. 全自动批处理 ====
def run_marker_roi_analysis(adata, marker_list, win_row=100, win_col=100, outdir="roi_analysis"):
    os.makedirs(outdir, exist_ok=True)
    all_stats = []
    for gene in marker_list:
        if gene not in adata.var_names:
            print(f"[跳过] {gene} 不在 adata.var_names 中")
            continue
        coords, score = find_high_expr_roi(adata, gene, win_row, win_col)
        r0, r1, c0, c1 = coords
        mask = ((adata.obs['array_row'] >= r0) & (adata.obs['array_row'] <= r1) &
                (adata.obs['array_col'] >= c0) & (adata.obs['array_col'] <= c1))
        adata_roi = adata[mask].copy()

        # bin_gdf
        bin_polygons = []
        half = (2 / 0.5) / 2
        for x, y in adata_roi.obsm['spatial']:
            poly = Polygon([
                (x-half, y-half), (x+half, y-half),
                (x+half, y+half), (x-half, y+half)
            ])
            bin_polygons.append(poly)
        bin_expr = pd.DataFrame(adata_roi.X.toarray(), columns=adata_roi.var_names)
        bin_gdf = gpd.GeoDataFrame(bin_expr, geometry=bin_polygons)

        # cell_gdf
        cell_ids = adata_roi.obs['labels_joint'].values
        coords_roi = adata_roi.obsm['spatial']
        unique_cells = np.unique(cell_ids[cell_ids > 0])
        cell_polys, valid_ids = [], []
        for cid in unique_cells:
            pts = coords_roi[cell_ids == cid]
            if len(pts) >= 3:
                cell_polys.append(make_valid(Polygon(pts).convex_hull))
                valid_ids.append(cid)
        cell_gdf = gpd.GeoDataFrame({"cell_id": valid_ids}, geometry=cell_polys)

        # 三方法
        pred_naive = naive_assignment(bin_gdf, cell_gdf)
        pred_wa = weighted_by_area(bin_gdf, cell_gdf)
        pred_sphere = sphere_assignment(bin_gdf, cell_gdf)

        # 绘图和统计
        outpath = os.path.join(outdir, f"{gene}_roi")
        num_cells, stats, improvement = plot_gene_roi_comparison_with_stats(
            cell_gdf, pred_naive, pred_wa, pred_sphere,
            gene, boundary_q=0.6, savepath=outpath
        )

        all_stats.append({
            "gene": gene,
            "roi_row_range": f"{r0}-{r1}",
            "roi_col_range": f"{c0}-{c1}",
            "roi_mean_expr": score,
            "boundary_cells": num_cells,
            **{f"{k}_mean": v for k, v in stats.items()},
            "SPHERE_vs_WA_%": improvement
        })

    pd.DataFrame(all_stats).to_csv(os.path.join(outdir, "roi_stats_HumanCRC.csv"), index=False)
    print(f"分析完成，结果保存在 {outdir}/")


In [3]:


# ===== 6. Human CRC marker genes =====
marker_list_crc = [
    "EPCAM", "KRT8", "KRT20", "MUC2",
    "CD3D", "CD3E", "CD8A",
    "CD79A", "MS4A1",
    "CD68", "LYZ",
    "ACTA2", "VIM", "COL1A1", "FAP",
    "MKI67", "TOP2A", "PCNA"
]

# ===== 7. 执行分析 =====
run_marker_roi_analysis(adata, marker_list_crc, win_row=100, win_col=100, outdir="HumanCRC_roi_analysis")


[跳过] KRT18 不在 adata.var_names 中
分析完成，结果保存在 HumanCRC_roi_analysis/


Figure 3 Legends（英文版）
Figure 3.
Representative ROI-based comparison of cell assignment methods in human colorectal cancer (CRC) VisiumHD data.

(a) ROI visualization for the top three marker genes with the largest improvement in mean expression within the ROI when using SPHERE compared with the Weighted Area method. For each marker, expression maps are shown for Naïve, Weighted Area, and SPHERE assignments, along with a difference map (SPHERE − Weighted Area). Red outlines indicate the convex hull enclosing the top 40% of cells by SPHERE-predicted expression.

(b) Bar plot showing the percentage improvement in ROI mean expression for SPHERE over Weighted Area across all CRC markers.

(c) Win-rate heatmap showing the percentage of markers for which each method achieved the highest ROI mean expression.



To further evaluate method performance in real human tissue, we applied Naïve, Weighted Area, and SPHERE assignments to VisiumHD human colorectal cancer (CRC) data. We focused on regions of interest (ROIs) centered on high-expression areas for biologically relevant marker genes. As shown in Figure 3a, for top-performing markers such as CD79A, CD3D, and MUC2, SPHERE yielded markedly higher expression estimates within the ROI compared with Weighted Area, particularly for gland- or immune-rich regions. The improvement was consistent across a spectrum of markers, with median ROI mean-expression gains exceeding 20% for most and reaching nearly 80% for the best-performing marker (Figure 3b). Win-rate analysis revealed that SPHERE prevailed over both Naïve and Weighted Area in the majority of markers (Figure 3c), underscoring its robustness for recovering biologically meaningful expression patterns in complex CRC tissue.



In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
def make_HumanCRC_figure3(outdir_roi, top_n_visual=5, fig3_prefix="Figure3_HumanCRC"):
    """
    Figure3：
      上部：ROI 面板 (a)
      下部：左 b 图（条形图，窄宽度）+ 右 c 图（散点图，窄宽度）
    同时输出 PNG 和 PDF
    """
    import matplotlib.gridspec as gridspec

    # 读取数据
    stats_path = os.path.join(outdir_roi, "roi_stats_HumanCRC.csv")
    stats_df = pd.read_csv(stats_path)
    stats_sorted = stats_df.sort_values("SPHERE_vs_WA_%", ascending=False)
    chosen_genes = stats_sorted.head(top_n_visual)["gene"].tolist()

    # 图布局：上面的 ROI 行跨整行，下面一行用 6 列宽度，其中 b,c 各占 2 列，中间留 2 列空白
    fig = plt.figure(figsize=(18, 4*top_n_visual + 6))
    gs = gridspec.GridSpec(top_n_visual + 1, 6, height_ratios=[4]*top_n_visual + [4])

    # ROI 面板 (a)
    for row_idx, gene in enumerate(chosen_genes):
        ax = fig.add_subplot(gs[row_idx, :])  # 跨所有列
        roi_img_path = os.path.join(outdir_roi, f"{gene}_roi.png")
        if os.path.exists(roi_img_path):
            img = plt.imread(roi_img_path)
            ax.imshow(img)
            ax.set_title(gene, fontsize=14, fontweight='bold')
        else:
            ax.text(0.5, 0.5, f"Missing ROI: {gene}", ha='center', va='center')
        ax.axis("off")
        if row_idx == 0:
            ax.text(-0.02, 1.02, "(a)", transform=ax.transAxes,
                    fontsize=14, fontweight='bold')

    # b 图：条形图（窄）
    ax_bar = fig.add_subplot(gs[-1, 0:2])  # 前两列
    sns.barplot(
        data=stats_sorted,
        x="SPHERE_vs_WA_%",
        y="gene",
        palette="viridis",
        ax=ax_bar
    )
    ax_bar.axvline(0, color="black", linewidth=1)
    ax_bar.set_xlabel("ROI Mean Expression Improvement (%)", fontsize=13)
    ax_bar.set_ylabel("")
    ax_bar.set_title("All Markers", fontsize=14)
    ax_bar.text(-0.2, 1.02, "(b)", transform=ax_bar.transAxes,
                fontsize=14, fontweight='bold')

    # c 图：Bland–Altman plot (SPHERE vs WA)
    mean_vals = (stats_df["SPHERE_mean"] + stats_df["Weighted Area_mean"]) / 2
    diff_vals = stats_df["SPHERE_mean"] - stats_df["Weighted Area_mean"]

    ax_ba = fig.add_subplot(gs[-1, 4:6])
    ax_ba.scatter(mean_vals, diff_vals, alpha=0.7, color="steelblue", edgecolors="k", s=50)

    # 添加均差线和一致性界限
    mean_diff = diff_vals.mean()
    sd_diff = diff_vals.std()
    ax_ba.axhline(mean_diff, color='red', linestyle='--', lw=1.2, label=f"Mean diff = {mean_diff:.2f}")
    ax_ba.axhline(mean_diff + 1.96*sd_diff, color='gray', linestyle=':', lw=1)
    ax_ba.axhline(mean_diff - 1.96*sd_diff, color='gray', linestyle=':', lw=1)

    ax_ba.set_xlabel("Mean ROI Expression (SPHERE & WA)", fontsize=13)
    ax_ba.set_ylabel("Difference (SPHERE − WA)", fontsize=13)
    ax_ba.set_title("Bland–Altman Plot", fontsize=14)
    ax_ba.text(-0.25, 1.02, "(c)", transform=ax_ba.transAxes,
            fontsize=14, fontweight='bold')
    ax_ba.legend()

    plt.tight_layout()

    # 输出 PNG 和 PDF
    png_path = f"{fig3_prefix}.png"
    pdf_path = f"{fig3_prefix}.pdf"
    plt.savefig(png_path, dpi=300)
    plt.savefig(pdf_path, dpi=300)
    plt.close()
    print(f"[完成] Figure 3 保存到 {png_path} 和 {pdf_path}")

In [5]:
make_HumanCRC_figure3(
    outdir_roi="HumanCRC_roi_analysis",
    top_n_visual=3,
    fig3_prefix="Figure3_HumanCRC"
)

117373726.py (41): 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



[完成] Figure 3 保存到 Figure3_HumanCRC.png 和 Figure3_HumanCRC.pdf
